In [28]:
import sys
print(sys.executable)
#!{sys.executable} -m pip install ase
#!{sys.executable} -m pip install tk
import tkinter
print(tkinter.Tcl().eval('info patchlevel'))


/Users/mat3ra/.pyenv/versions/3.11.4/bin/python3.11
8.6.13


In [11]:
from ase.visualize import view
import ase.build
show = False

crystal = ase.build.bulk("Ni", cubic=True)
if show : view(crystal*[4,4,1])

graphene = ase.build.graphene()
if show : view(graphene*[2,2,1])


In [129]:
# data
global data
substrate_poscars = ["""Ni4
1.0
   3.4751458659480110    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.4751458659480110    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.4751458659480110
Ni
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Ni
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Ni
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Ni
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Ni
""",
"""Cu4
1.0
   3.5774306715697510    0.0000000000000000    0.0000000000000002
   0.0000000000000006    3.5774306715697510    0.0000000000000002
   0.0000000000000000    0.0000000000000000    3.5774306715697510
Cu
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Cu
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Cu
""",
"""Au4
1.0
   4.1712885314747270    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.1712885314747270    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.1712885314747270
Au
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Au
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Au
""",
"""Si4 C4
1.0
   4.3539932475828609    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.3539932475828609    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.3539932475828609
Si C
4 4
direct
   0.7500000000000000    0.2500000000000000    0.7500000000000000 Si4+
   0.7500000000000000    0.7500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.2500000000000000    0.2500000000000000 Si4+
   0.2500000000000000    0.7500000000000000    0.7500000000000000 Si4+
   0.0000000000000000    0.0000000000000000    0.0000000000000000 C4-
   0.0000000000000000    0.5000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.0000000000000000    0.5000000000000000 C4-
   0.5000000000000000    0.5000000000000000    0.0000000000000000 C4-
"""]

material_poscars = ["""Graphene
1.0
   2.467291000	   0.000000000	   0.000000000
  -1.233645000	   2.136737000	   0.000000000
   0.000000000	   0.000000000	   7.803074000
C
2
direct
   0.000000000    0.000000000    0.000000000  C
   0.333333000    0.666667000    0.000000000  C
""",
"""Au4
1.0
   4.1712885314747270    0.0000000000000000    0.0000000000000003
   0.0000000000000007    4.1712885314747270    0.0000000000000003
   0.0000000000000000    0.0000000000000000    4.1712885314747270
Au
4
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Au
   0.0000000000000000    0.5000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.0000000000000000    0.5000000000000000 Au
   0.5000000000000000    0.5000000000000000    0.0000000000000000 Au
"""]

substrate_poscar = substrate_poscars[0]
material_poscar = material_poscars[0]
data = [substrate_poscar, material_poscar]

In [149]:
# settings
global settings 
settings = {"slab_settings": {"h":1, "k":1, "l":0, "vacuum":0.5}, 
            "interface_settings": {"substrate":{"n":1, "m":2}, "material":{"n": 3, "m": 4}}, 
            "supercell_settings":{"n":3, "m":3}}

In [150]:
from ase import Atoms
from ase.build import surface
from ase.io import read, write
import io
from operator import itemgetter

def poscar_to_atoms(poscar):
    input = io.StringIO(poscar)
    atoms = read(input, format="vasp")
    return atoms
    
def write_atoms_to_poscar(atoms) :
    output = io.StringIO()
    write(output, atoms, format="vasp")
    content = output.getvalue()
    output.close()
    return content

class MaterialInterface:
    def __init__(self, substrate, material, settings=None):
        self.substrate = substrate
        self.material = material
        self.settings = {
            "slab_settings": {"h": 1, "k": 0, "l": 0, "layers": 3, "vacuum": 0.5},
            "interface_settings": {"substrate":{"n":1, "m":1}, "material": {"n":1, "m":1}, "distance": 2.0},
            "supercell_settings": {"n":1, "m":1}
        }
        if settings:
            for key in self.settings.keys():
                if key in settings:
                    self.settings[key].update(settings[key])
        self.structure = self.create_structure()
    
    def create_structure(self):
        h, k, l, layers, vacuum = itemgetter('h', 'k', 'l', 'layers','vacuum')(self.settings["slab_settings"])
        
        distance = itemgetter('distance')(self.settings["interface_settings"])
        substrate_supercell = self.settings["interface_settings"]["substrate"]
        material_supercell = self.settings["interface_settings"]["material"]
        
        slab = surface(self.substrate, (h, k, l), layers, vacuum)
        slab = slab * [substrate_supercell["n"], substrate_supercell["m"], 1]
        material = self.material * [material_supercell["n"], material_supercell["m"], 1]

        z_max_slab = max(slab.positions[:, 2])
        z_min_material = min(material.positions[:, 2])
        z_offset = z_max_slab - z_min_material + distance
        material.positions[:, 2] += z_offset

        return slab + material
        
    def create_supercell(self):
        n, m = itemgetter("n", "m")(self.settings["supercell_settings"])
        self.structure = self.structure * [n, m, 1]

    def calculate_strain():
        return "not implemetned"
        
    def view(self):
        view(self.structure)

def func() :
    """This function is meant to be a gateway to Pyodide in Materials Designer"""
    
    poscars = globals()["data"]
    settings = globals()["settings"]
    materials = []
    for poscar in poscars :
        print(poscar_to_atoms(poscar))
        materials.append(poscar_to_atoms(poscar))

    
    interface = MaterialInterface(materials[0], materials[1], settings)
    write("structure.poscar", interface.structure, format='vasp')
    
    interface.create_supercell()
    view(interface.structure)

func()


Atoms(symbols='Ni4', pbc=True, cell=[[3.475145865948011, 0.0, 2e-16], [6e-16, 3.475145865948011, 2e-16], [0.0, 0.0, 3.475145865948011]])
Atoms(symbols='C2', pbc=True, cell=[[2.467291, 0.0, 0.0], [-1.233645, 2.136737, 0.0], [0.0, 0.0, 7.803074]])
